In [ ]:
!pip install crc64iso

In [ ]:
from crc64iso import crc64iso
import pandas as pd
from pathlib import Path

In [ ]:
fireprot_path = Path('/kaggle/input/fireprot-external-dataset')
fireprot = pd.read_csv(fireprot_path/'csv/fire.csv')
fireprot = fireprot[fireprot.pdb_id_2.isnull()]

firecols = ['pdb_path_1', 'wildtype', 'position', 'mutation', 'ddG',
            'wildtype_sequence', 'mutant_sequence', 
            'dTm', 
           ]

fireprot = fireprot[firecols]

fireprot = fireprot.rename(columns={'wildtype_sequence': 'sequence',
                         'mutant_sequence': 'mutant_seq',
                         'pdb_path_1': 'PDB'})

fireprot['PDB'] = fireprot['PDB'].map(lambda x: Path(x).name)

In [ ]:
crc64s = "','".join(pd.Series(fireprot.sequence.unique()).apply(lambda v: crc64iso.crc64(str(v))))
sql = f"""
SELECT
  sequenceChecksum,
  entryID,
  CONCAT('gs://public-datasets-deepmind-alphafold-v4/',entryID, '-model_v', latestVersion, '.cif') as cif,
  CONCAT('gs://public-datasets-deepmind-alphafold-v4/',entryID, '-confidence_v', latestVersion, '.json') as confidence,
  CONCAT('gs://public-datasets-deepmind-alphafold-v4/',entryID, '-predicted_aligned_error_v', latestVersion, '.json') as err
FROM bigquery-public-data.deepmind_alphafold.metadata
WHERE sequenceChecksum in ('{crc64s}')
"""
print(sql[:200])

In [ ]:
with open('metadata_fireprot_wildtypes.sql', 'w+') as f:
    f.write(sql)

Use BigQuery:

https://console.cloud.google.com/bigquery?sq=264715149975:fb373d507b5645f3a7543f1e19315807&project=powerful-anchor-228512&ws=!1m9!1m4!1m3!1spowerful-anchor-228512!2sbquxjob_4322957e_184ce4bdfe3!3sUS!1m3!8m2!1s264715149975!2sfb373d507b5645f3a7543f1e19315807

In [ ]:
fireprot_crc64_cif = pd.read_csv('/kaggle/input/fireprot-crc64-cif/fireprot_wildtypes_crc64_cif.csv')
fireprot_crc64_cif

In [ ]:
cifs = ' '.join(fireprot_crc64_cif.cif)

## Run locally following command 

> ! gsutil cp  $cifs /